# 学習に関するテクニック

In [1]:
import sys, os
sys.path.append('deep-learning-from-scratch')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from common import 

%matplotlib inline

SyntaxError: invalid syntax (<ipython-input-1-db87a446c3cb>, line 8)

## パラメータの更新

### 冒険家の例え話 (の解釈)

広い荒野の中で一番深い谷底を目指す冒険家。  
しかし彼は目が見えない。  
どうやったら彼は効率的に深い谷底へたどり着くことができるだろうか。 (BGM: 広野を行く)



###  SGD (確率的勾配降下法 stochastic gradient descent)

重みパラメータWを次に `W - 学習率*δL/δW`に更新していく。



In [ ]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
    
    def update(self, param, grads):
        for key in params.keys():
            param[key] -= self.lr * grads[key]

lr: learning rate.

#### SGDの欠点
関数の形状が等方的でなく、伸びた関数だとジグザグに学習が進み、とても非効率な探索をする傾向にある。

TODO:// 実際に描画してみたい。

### Momentum

モーメンタム: 運動量

- v_n+1 = αv_n - η(δL/δW)
- W_n+1 = W_n + v_n+1


各変数の役割

- v: 速度
- α: 徐々に減速させるための定数。0.9等の値を入れる。空気抵抗のようなもの。


In [ ]:
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momenmum
        self.v =None
    
    def update(self, param, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
        
        for key in params.keys():
            self.v[key] = self.momentum * self.v[key] - self.lr * grads[key]
            param[key] += self.lr * grads[key]

### AdaGrad

NNでは学習係数ηが小さすぎると学習に時間がかかり、大きすぎると発散して正しい学習が行えない。
学習係数に関する有効なテクニックとして、学習係数の減衰(learning rate decay)という方法が使われる。

`AdaGrad`はパラメータの要素ごとに適応的に学習係数を調整しながら学習を行う手法。(Ada: Adaptive)

- h_n+1 = h_n + ((δL/δW) ・(δL/δW))
- W_n+1 = W_n - η(1/√h+1 * δL / δW)

これまで経験した勾配の値を二乗和として保持し、よく動いた要素は学習係数が小さくなるような数式。


In [2]:
class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h =None
    
    def update(self, param, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
        
        for key in params.keys():
            self.h[key] = grads[key] * grads[key]
            param[key] -= self.lr * grads[key] / (np.sqrt(self.h[key])) + 1e-7

### Adam
Momentum + AdaGrad
効率的にパラメータ空間を探索できる。
また、ハイパーパラメータの「バイアス補正」が行われていることも特徴


### まとめ

SGDよりも他の3手法のほうが早く学習が進む傾向があるが、どれが一番いいとは一概には言えない。

## 重みの初期値の設定の仕方

### 隠れ層のアクティベーション分布

隠れ層のアクティベーション(活性化関数の後のデータ)の分布が、重みの初期値によってどのように変化するかの実験を見ていく。

アクティベーションに偏りがあると、表現力が制限されてしまう。

### Xavierの初期値

データがn子あったときに、1 / √n の標準偏差を持つ分布。

活性化関数が線形であることが前提。Sigmoid関数やTanhは左右対称で中央付近が線形関数としてみなすことが可能。


### ReLUの場合の重み初期値: He

前層のノード数がnの場合、√ n/2 を標準偏差とするガウス分布

## Batch Normalization

2015年に提案された手法にも関わらず、現在多くの場所で使われている

- 学習を早く進行させることができる
- 初期値にあまり依存しない
- 過学習を抑制する

データ分布の正規化を行うBatchNormalizationレイヤを、NNに追加する

## 正則化

### 過学習

- パラメータを大量に持ち、表現力の高いモデル
- 訓練データが少ない

というような要件を満たすと、過学習が起こりやすい

### Weight decay(荷重減衰)

過学習を抑制する手法

L2ノルムを加算する、Weight decay
`(1/2)λW**2` (λが大きいほど、大きな値へのペナルティが大きくなる)

W = (w1, w2, ...wn)があるとき L2ノルム = √(w1^2 + w2^2 + ... + wn^2)

### dropout

ニューロンをランダム消去しながら学習する。

- 訓練時: 隠れ層のニューロンをランダムに選び出して、消去
- テスト時: すべてのニューロンに信号を伝達するが、各ニューロンの出力に対して訓練時に消去した割合を乗算して出力

In [3]:
class Dropout:
    def __init__(self, dropout_ratio=0.5):
        self.dropout_ratio = dropout_ratio
        self.mask = None
    
    def forward(self, x, train_flg=True):
        if train_flg:
            self.mask = np.random.rand(*x.shape) > self.drop_ratio
            return x * self.mask
        return x * (1.0 - dropout_ratio)
    
    def backward(self, dout):
        return dout * self.mask
            

## ハイパーパラメータの検証と調整

### ハイパーパラメータの最適化

1. ハイパーパラメータの範囲を設定
2. 設定された範囲からランダムにサンプリング3
3. 1. でサンプリングされたパラメータを利用して、エポックを小さめにした検証データで認識精度を評価する
4. 1.2.をある程度(100回とか)繰り返して、認識精度の結果からパラメータの範囲を狭め得る

## まとめ

- パラメータの更新方法SGD/Momentum. AdaGrad, Adamを学んだ
- 重みの初期値として、Xavierの初期値やHeの初期値(ReLU専用)等が有効
- Batch Normalizationを用いることで、学習を早くすすめることができる
- 過学習を抑制するための正則化の技術として、weight decayやdropoutがある
- ハイパーパラメータは範囲をだんだん狭めながらいいは安易を探す